# Lab 2

The aims of the lab are to:

*   Perform tokenization and stemming of text using NLTK
*   Learn basics of cleaning ‘noisy’ text
*   Calculate and visualize basic collection statistics of a corpus of text
*   Practice using a bag-of-words using a one-hot encoding
*   Learn to use SciKit learn to extract term frequency and TF-IDF features
*   Use SciKit learn to ‘find similar’ documents using Cosine similarity
*   Perform KMeans clustering on posts
*   Learn to perform basic evaluation of clusters through manual inspection
*   Use clustering to explore a corpus



## Reddit Data Revisited

The data is provided to download below. You can read more about the origins of the data at:
https://github.com/google-research-datasets/coarse-discourse

**Thread fields**
*   URL - reddit URL of the thread
*   title - title of the thread, as written by the first poster
*   is_self_post - True if the first post in the thread is a self-post (text addressed to the reddit community as opposed to an external link)
*   subreddit - the subreddit of the thread
*   posts - a list of all posts in the thread

**Post fields**
*   id - post ID, reddit ID of the current post
*   in_reply_to - parent ID, reddit ID of the parent post, or the post that the current post is in reply to
*   post_depth - the number of replies the current post is from the initial post
*   is_first_post - True if the current post is the initial post
*   annotations - a list of all annotations made to this post (see below)
*   majority_type - the majority annotated type, if there is a majority type between the annotators, when considering only the main_type field
*   majority_link - the majority annotated link, if there is a majority link between the annotators


Download the Reddit dataset
<Insert reddit data description>
  

In [3]:
# The local location to store the reddit dataset.
local_file = "coarse_discourse_dump_reddit.json"

# The ! performs a shell command to download the reddit dataset using curl (not on Windows).
!curl -o  $local_file https://storage.googleapis.com/tad2018/coarse_discourse_dump_reddit.json

#!gsutil cp gs://tad2018/coarse_discourse_dump_reddit.json /tmp/coarse_discourse_dump_reddit.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 78.5M  100 78.5M    0     0  5125k      0  0:00:15  0:00:15 --:--:-- 5587k14 11.4M    0     0  5386k      0  0:00:14  0:00:02  0:00:12 5386k  4965k      0  0:00:16  0:00:14  0:00:02 3404k


Let's extract the post content out into a posts global data frame that we'll use for our processing.

In [4]:
# The reddit thread structure is nested with posts in a new content.
# This block reads the file as json and creates a posts data frame.
import pandas as pd
import json

# A temporary variable to store the list of posts.
posts_tmp = list()

with open(local_file) as jsonfile:
  for i, line in enumerate(jsonfile):
    thread = json.loads(line)
    for post in thread['posts']:
      # Keep the thread title, subreddit, and url with each post.
      posts_tmp.append((thread['subreddit'], thread['title'], thread['url'],
                        post['id'], post.get('author', ""), post.get('body', "")))
print(len(posts_tmp))

# Create the posts data frame with the right column labels.  
labels = ['subreddit', 'title', 'id', 'url', 'author', 'body']
post_frame = pd.DataFrame(posts_tmp, columns=labels)

110595


## Tokenization and stemming with NLTK

Let's perform some basic tokenization.

In [5]:
# NLTK is the Natural Language Toolkit, and contains several language datasets
# as well as implementations of many popular NLP algorithms.
# HINT: You should look at what is available here when thinking about your coursework!
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer

import matplotlib.pyplot as plt
import math

# A simple tokenizer based on a regular expression; a series of whitespace chars.
tokenizer = RegexpTokenizer(r'\w+')

We first create a basic 'normalization' or 'canonicalization' function that converts text into a standard format. The simple function performs tokenization and lowercasing. It uses the NLTK regular expression tokenizer.

In [6]:
def basic_canonicalize(string):
  normalized_tokens = list()
  tokens = tokenizer.tokenize(string)
  for t in tokens:
    # Normalizes the token by lowercasing it.
    normalized_tokens.append(t.lower())
  return normalized_tokens

Now that we have our tokenizer, let's use it to start exploring the collection. 


In [7]:
import itertools

# This tokenizes the body posts and creates vector of tokens for each post.
all_posts_tokenized = post_frame.body.apply(basic_canonicalize)

# A single variable to hold the tokens across all posts.
all_tokens = list(itertools.chain.from_iterable(all_posts_tokenized))

# Put the entire array of tokens into a NLTK Frequency Distribution class.
word_dist = nltk.FreqDist(all_tokens)



**Exercise**: 

*   How many posts are empty?
*   What is the total number of words?
*   What is the average length of the post?


Use the API of NLTK's [FreqDist](http://www.nltk.org/api/nltk.html?highlight=freqdist#nltk.probability.FreqDist) class to calculate the total number of words, the average length of a post. Using FreqDist for this may be overkill here, but it's a useful class for modeling language that we'll see next week.

In [8]:
counter=0
for post in all_posts_tokenized:
    if len(post)==0:
        #print(post)
        counter=counter+1
print(counter)

2642


**Exercise**: Print out the 50 most frequent words in the collection, and their frequency using the FreqDist API. Hint: FreqDist extends [collections.Counter](https://docs.python.org/2/library/collections.html#collections.Counter).

In [9]:
word_dist.most_common(50)

[('the', 176166),
 ('i', 146882),
 ('to', 124121),
 ('a', 113949),
 ('and', 102499),
 ('it', 83866),
 ('you', 77818),
 ('of', 73506),
 ('that', 61859),
 ('is', 60014),
 ('in', 56321),
 ('for', 48310),
 ('s', 40905),
 ('t', 39430),
 ('on', 34514),
 ('but', 34365),
 ('with', 33206),
 ('have', 33180),
 ('be', 32589),
 ('this', 29944),
 ('my', 29500),
 ('if', 28520),
 ('are', 25468),
 ('not', 25461),
 ('as', 25383),
 ('can', 23862),
 ('or', 23414),
 ('was', 23357),
 ('so', 22401),
 ('they', 22291),
 ('just', 21495),
 ('your', 20866),
 ('like', 20160),
 ('at', 18739),
 ('what', 16815),
 ('do', 16595),
 ('would', 16367),
 ('there', 16150),
 ('m', 16050),
 ('me', 15910),
 ('get', 15712),
 ('all', 15610),
 ('he', 15354),
 ('from', 14850),
 ('one', 14605),
 ('out', 14295),
 ('about', 14281),
 ('will', 14073),
 ('up', 14037),
 ('don', 13540)]

What do you see?  The most common words are functional words, often referred to as 'stopwords' because they contain little information.  We should clean the data by removing these 'noise' words that don't have significant meaning on their own.

We need a 'real' tokenizer that does stopping, stemming, and filters out other 'noise' words. It just so happens that NLTK has a built in stopword list. Download it and use it.

In [10]:
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/motoofi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Exercise:** Create a function, `canonicalize` that filters 'meaningless' words.

Modify the function to filter out 'noise' words. Use the NLTK Porter stemmer to stem the text.
*   What about 'stop' words?
*   What about excessively short tokens or 'long' tokens?

Hint: Does the order matter? What could happen if stemming is applied after stopping?


In [11]:
import numpy as np
tokens = word_dist.keys()
lengths = []
for token in tokens:
    lengths.append(len(token))
std = np.std(lengths)
mean = np.mean(lengths)
print("mean{0}".format(mean))
print("std:{0}".format(std))
print("max:{0}".format(max(lengths)))

mean8.02716631884584
std:7.798074686968009
max:1440


In [18]:
# Create an instance of the NLTK Porter Stemmer discussed in lecture to use.
stemmer = PorterStemmer()

# Modify the code below to 1) not append noisy words: words in stop_words or words that are too long or too short.
# Question: What are good values for too long or too short? Why?
def canonicalize(string):
  normalized_tokens = list()
  tokens = tokenizer.tokenize(string)
  for token in tokens:
    # YOUR CODE HERE
    # Ignore stop words and other 'noise' words (too short? too long?)
    # Lowercase and stem the words.
    # Does the order of stopping and stemming matter? 
    #if (token in stop_words) or ((mean-std)<len(token)<(mean+std)) :
    normalized = token.lower()
    stem_and_normilized = stemmer.stem(normalized)
    if (stem_and_normilized in stop_words):
        pass
    elif (len(stem_and_normilized)<(mean+std)) and (mean-std<len(stem_and_normilized)):
        normalized_tokens.append(stem_and_normilized)
  return normalized_tokens

Now, let's repeat our exercise.  How have the statistics changed?


In [20]:
# This tokenizes the body posts and creates vector of tokens for each post.
all_posts_tokenized = post_frame.body.apply(canonicalize)

# A single variable to hold the tokens across all posts.
all_tokens = list(itertools.chain.from_iterable(all_posts_tokenized))

# Put the entire array of tokens into a NLTK Frequency Distribution class.
word_dist = nltk.FreqDist(all_tokens)


Use your code for most frequent words discussed above here to print out the most frequent 50 words again.

In [21]:
word_dist.most_common(50)

[('thi', 29957),
 ('wa', 23377),
 ('like', 22210),
 ('get', 20548),
 ('would', 16367),
 ('one', 15873),
 ('http', 15796),
 ('go', 14118),
 ('use', 13954),
 ('com', 13199),
 ('time', 12666),
 ('think', 12474),
 ('make', 11450),
 ('know', 10671),
 ('realli', 10655),
 ('game', 10593),
 ('want', 10370),
 ('good', 10328),
 ('ha', 9977),
 ('peopl', 9729),
 ('also', 9548),
 ('work', 9278),
 ('look', 9177),
 ('becaus', 8956),
 ('thing', 8920),
 ('onli', 8770),
 ('ani', 8708),
 ('tri', 8512),
 ('much', 8295),
 ('need', 8157),
 ('play', 7873),
 ('see', 7862),
 ('could', 7492),
 ('way', 7429),
 ('2', 7231),
 ('hi', 7191),
 ('even', 7170),
 ('well', 7140),
 ('1', 7097),
 ('thank', 6904),
 ('www', 6889),
 ('say', 6813),
 ('take', 6544),
 ('veri', 6269),
 ('still', 6142),
 ('lot', 6126),
 ('someth', 6019),
 ('year', 6014),
 ('3', 5919),
 ('start', 5785)]

Looking at this list, it still contains reddit-specific words that may not be meaningful. For example, 'http'? Update the stop word set and repeat the above block until you are happy with the list.

**Exercise: ** Modify the stop_words set to include additional noise words. Re-run the posts tokenization and top word calculation.



In [22]:
stop_words.update(('http', ))
all_posts_tokenized = post_frame.body.apply(canonicalize)
all_tokens = list(itertools.chain.from_iterable(all_posts_tokenized))
word_dist = nltk.FreqDist(all_tokens)
word_dist.most_common(50)

[('thi', 29957),
 ('wa', 23377),
 ('like', 22210),
 ('get', 20548),
 ('would', 16367),
 ('one', 15873),
 ('go', 14118),
 ('use', 13954),
 ('com', 13199),
 ('time', 12666),
 ('think', 12474),
 ('make', 11450),
 ('know', 10671),
 ('realli', 10655),
 ('game', 10593),
 ('want', 10370),
 ('good', 10328),
 ('ha', 9977),
 ('peopl', 9729),
 ('also', 9548),
 ('work', 9278),
 ('look', 9177),
 ('becaus', 8956),
 ('thing', 8920),
 ('onli', 8770),
 ('ani', 8708),
 ('tri', 8512),
 ('much', 8295),
 ('need', 8157),
 ('play', 7873),
 ('see', 7862),
 ('could', 7492),
 ('way', 7429),
 ('2', 7231),
 ('hi', 7191),
 ('even', 7170),
 ('well', 7140),
 ('1', 7097),
 ('thank', 6904),
 ('www', 6889),
 ('say', 6813),
 ('take', 6544),
 ('veri', 6269),
 ('still', 6142),
 ('lot', 6126),
 ('someth', 6019),
 ('year', 6014),
 ('3', 5919),
 ('start', 5785),
 ('first', 5750)]

We will now look at how we represent words using a one-hot encoding in more detail.

Look at the Vocabulary class below:
- What does it do? 
- How are words mapped to numbers? 
- How are the words ordered? Why might we want to do this?
- What is the 'unk' token? And when is it used?
- What happens if you perform words_to_ids on a word not in the original tokens?
- Why might you want special start and end tokens?


In [23]:
import collections

class Vocabulary(object):

  START_TOKEN = "<s>"
  END_TOKEN = "</s>"
  UNK_TOKEN = "<unk>"

  def __init__(self, tokens, size=None):
    # Counter is a very useful built-in python collection for keeping counts, 
    # Instead of extending Counter like FreqDist, it's used as a member variable.
    self.unigram_counts = collections.Counter(tokens)
    self.num_unigrams = sum(iter(self.unigram_counts.values()))
    # leave space for "<s>", "</s>", and "<unk>"
    top_counts = self.unigram_counts.most_common(None if size is None else (size - 3))
    vocab = ([self.START_TOKEN, self.END_TOKEN, self.UNK_TOKEN] +
             [w for w,c in top_counts])

    # Assign an id to each word, by frequency.
    self.id_to_word = dict(enumerate(vocab))
    self.word_to_id = {v:k for k,v in iter(self.id_to_word.items())}
    self.size = len(self.id_to_word)
    if size is not None:
        assert(self.size <= size)

    # For convenience keep a set of unique words.
    self.wordset = set(iter(self.word_to_id.keys()))

    # Store special IDs.
    self.START_ID = self.word_to_id[self.START_TOKEN]
    self.END_ID = self.word_to_id[self.END_TOKEN]
    self.UNK_ID = self.word_to_id[self.UNK_TOKEN]

  def words_to_ids(self, words):
    return [self.word_to_id.get(w, self.UNK_ID) for w in words]

  def ids_to_words(self, ids):
    return [self.id_to_word[i] for i in ids]

  def sentence_to_ids(self, words):
    return [self.START_ID] + self.words_to_ids(words) + [self.END_ID]

  def ordered_words(self):
    """Return a list of words, ordered by id."""
    return self.ids_to_words(range(self.size))

**Exercise:** Use the Vocabulary class to print out the top 10 most frequent unigrams.

In [29]:
vocab = Vocabulary(word_dist)
x=vocab.ordered_words()
x[0:9]

['<s>', '</s>', '<unk>', 'thi', 'wa', 'like', 'get', 'would', 'one']

Below are some example uses of the Vocabulary object to map words to ids and vice-versa. Execute the code and make sure you understand what is happening with the Vocabulary.



In [30]:
# Use our vocabulary to map words to integer ids.
# You should be able to predict the index of like based on your knowledge
# the Vocabulary class and the most frequent words.
print(vocab.words_to_ids(["like"]))

# What index should this return?
print(vocab.words_to_ids(["likemymadeupword"]))


# What's special about 2? Can you predict what it will be?
print(vocab.ids_to_words([2]))


# And let's print out the words from the vocab.
import random as rand

# Pick a random token in the vocabulary
idx = rand.randint(0, vocab.size-1)
print(idx, vocab.ids_to_words([idx]))

[5]
[2]
['<unk>']
72326 ['x4m6']


## Vector representations with Scikit-Learn

There are lots of ways to do the same thing in Python. Scikit-learn is a widely machine learning library that includes tools for performing operations on data: similarity computation, clustering, classification, and many others. We'll use Scikit-learn to create a vector representations of our text data.

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

# Parallel arrays of the post keys and values.
post_vals = list()
post_keys = list()

for index, post in post_frame.iterrows():
    post_keys.append(post['id'])
    post_vals.append(post['body'])


Create a simple TF term-document matrix with the TF counts using the [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html).

In [36]:
# We pass in our function tokenizer to the vectorizer object.
#creating an CountVectorizer obj
tf_vectorizer = CountVectorizer(tokenizer=canonicalize)

# This creates a vocabulary like our Vocabulary object above.
#create vocab for out corpera
tf_vectorizer.fit(post_vals)

# Now we create a sparse term-document matrix using the vocabulary.
# This performs the mapping of tokens to their IDs.
#make a representation for corpora
tf_term_document_matrix = tf_vectorizer.transform(post_vals)


# Note: These can be combined with fit_transform to do this in a single step.

<1x82749 sparse matrix of type '<class 'numpy.int64'>'
	with 183 stored elements in Compressed Sparse Row format>

**Exercise**: Type in a string below. Look at it's vector representation. What happens to 'unk' words?

In [52]:
str = 'salam khobi? I am fine thanks. Mersi I am fine too.'
response = tf_vectorizer.transform([str])
print (response)
print (tf_vectorizer.inverse_transform(response))

  (0, 31368)	2
  (0, 63701)	1
  (0, 72092)	1
[array(['fine', 'salam', 'thank'], dtype='<U15')]


The first step transforms our sentence using the vocbulary, creating a sparse TF vector representing the sentence.
The second operation allows us to see what words correspond to each of those elements. 

**Exercise:** Let's now upgrade from simple counts to a vectorizer that uses IDF. See the [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) API.  The TF-IDF Vectorizer uses the raw term frequency by default, use the version of the constructor that uses the log(tf).

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Import TfidfVectorizer from sklearn and fill in the code below:
tfidf_vectorizer = TfidfVectorizer(tokenizer=canonicalize ,sublinear_tf=True)
tfidf_vectorizer.fit(post_vals)
tfidf_term_document_matrix = tfidf_vectorizer.fit_transform(post_vals)


TF-IDF vectorizer also includes support for creating n-grams during tokenization. Check the documentation for TfidfVectorizer and create an instance that includes word bigram tokens.

In [55]:
# Limit the size of the vocabulary to the N most common words.
num_features=500000

ngram_vectorizer = TfidfVectorizer(max_features=num_features ,ngram_range=(2,2),sublinear_tf=True)
ngram_vectorizer.fit(post_vals)
ngram_term_document_matrix = ngram_vectorizer.fit_transform(post_vals)

Compare the n-gram representation of your string using the ngram vectorizer.

In [56]:
ngram_matrix = ngram_vectorizer.transform([str])
print (ngram_matrix)
print (ngram_vectorizer.inverse_transform(ngram_matrix))

  (0, 134752)	0.42977123093042935
  (0, 134742)	0.4591820891496106
  (0, 15936)	0.7774628596072888
[array(['fine too', 'fine thanks', 'am fine'], dtype='<U481')]


We're just scratching the surface of what's possible with SKLearn's vectorizers.  There are other types of representations as well. For example, there is the [HashingVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html) discussed in lecture as well others.

## Cosine similarity
We will now use sklearn's cosine similarity implementation to find the most similar posts.

In [57]:
from sklearn.metrics.pairwise import cosine_similarity
 
# A function that given an input query item returns the top-k most similar items 
# by their cosine similarity.
def find_similar(query_vector, td_matrix, top_k = 5):
    cosine_similarities = cosine_similarity(query_vector, td_matrix).flatten()
    sorted_indices = cosine_similarities.argsort()[::-1]
    return [(index, cosine_similarities[index]) for index in sorted_indices][0:top_k]

**Exercise:** Next, lets pick a post to compute similarity with. Put the content of a post in the variable called `str`

In [58]:
# Create an input 'query' string to find a similar post.
query = "machine learning"


In [62]:
# Temporary variables to easily change the vectorizer and the document_matrix.
vectorizer = ngram_vectorizer
document_matrix = ngram_term_document_matrix


# Below are options to use an existing post content as a string query.
#import random as rand
#post_index = 1000
#post_index = rand.randint(0, len(post_vals))
#str = post_vals[post_index]
#print(str.replace('\n', ''))

# Transform our string using the vocabulary.
transformed = vectorizer.transform([query])
query_transformed = transformed[0:1]
print("query:{0}".format(query_transformed))

top_k = 10
print ("\nSimilar:")
for index, score in find_similar(query_transformed, document_matrix, top_k):
  print(score, index, post_keys[index], post_vals[index].replace('\n', ''))

query:  (0, 204156)	1.0

Similar:
0.30901671388901053 43311 https://www.reddit.com/r/compsci/comments/2qdyks/good_machine_learning_graduate_grad_labs/ Machine learning is distinct from AI, and both are distinct from robotics, as far as I'm aware, though of course there is overlap (many consider machine learning to be a branch of AI). Not really sure whether you're asking about ML, robotics, or ML applied to robotics?
0.27281262895281994 82506 https://www.reddit.com/r/pebble/comments/3iokgu/why_do_i_have_to_send_usage_logs_to_pebble_just/ It's also easier to do the machine learning on the service side. Otherwise you're storing a TON of data locally on the device.
0.23198046200964764 43300 https://www.reddit.com/r/compsci/comments/2qdyks/good_machine_learning_graduate_grad_labs/ I'm looking for recommendations for graduate programs and laboratories that have experience with machine learning and AI, particularly related to robotics or cyber physical applications. I have a mech.eng BS, sig

What do you see?  If you use the post, it should find itself and return a similarity score of 1.0.

Try changing the vectorizers and matrix representations to the ones you constructed (count, tfidf, or ngrams). 
How do the most similar posts change?

## KMeans clustering

What's in the reddit dataset? When we want to explore a dataset, one way to that is to cluster the data.

As discussed in lecture, we'll now use an unsupervised clustering algorithm to cluster our posts.


From the SKlearn documentation:
*The [KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) algorithm clusters data by trying to separate samples in n groups of equal variance, minimizing a criterion known as the inertia or within-cluster sum-of-squares. This algorithm requires the number of clusters to be specified. It scales well to large number of samples and has been used across a large range of application areas in many different fields.*

*K-means is often referred to as Lloyd’s algorithm. In basic terms, the algorithm has three steps. The first step chooses the initial centroids, with the most basic method being to choose k samples from the dataset X. After initialization, K-means consists of looping between the two other steps. The first step assigns each sample to its nearest centroid. The second step creates new centroids by taking the mean value of all of the samples assigned to each previous centroid. The difference between the old and the new centroids are computed and the algorithm repeats these last two steps until this value is less than a threshold. In other words, it repeats until the centroids do not move significantly.*


In [63]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans

num_clusters = 20

# Note that there are multiple implementation of KMeans in sklearn. 
# The minibatchkmeans is faster, but less stable than the original kmeans.
# It's faster, but produces different clusterings.
#kmeans = MiniBatchKMeans(n_clusters=num_clusters, batch_size=500,  max_no_improvement=3)


kmeans = KMeans(n_clusters=num_clusters, n_init=2, verbose=10)
kmeans.fit(document_matrix)

Initialization complete
Iteration  0, inertia 102935.694
Iteration  1, inertia 102750.562
Iteration  2, inertia 102715.011
Iteration  3, inertia 102699.774
Iteration  4, inertia 102693.206
Iteration  5, inertia 102687.509
Iteration  6, inertia 102684.441
Iteration  7, inertia 102681.934
Iteration  8, inertia 102678.603
Iteration  9, inertia 102671.648
Iteration 10, inertia 102663.218
Iteration 11, inertia 102658.799
Iteration 12, inertia 102657.232
Iteration 13, inertia 102656.508
Iteration 14, inertia 102655.504
Iteration 15, inertia 102648.863
Iteration 16, inertia 102635.023
Iteration 17, inertia 102629.888
Iteration 18, inertia 102629.105
Iteration 19, inertia 102628.889
Iteration 20, inertia 102628.757
Iteration 21, inertia 102628.710
Iteration 22, inertia 102628.677
Iteration 23, inertia 102628.648
Iteration 24, inertia 102628.614
Iteration 25, inertia 102628.586
Iteration 26, inertia 102628.529
Iteration 27, inertia 102628.393
Iteration 28, inertia 102628.311
Iteration 29, inert

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=20, n_init=2, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=10)

We should now have a kmeans clustering with cluster centers, called 'centroids'. We'll print out the top 10 terms from each of the centroids. This will help us understand what each cluster represents.

In [64]:
centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
tokens = vectorizer.get_feature_names()
for i in range(num_clusters):
  print("Cluster %d:" % i)
  for ind in centroids[i, :10]:
    print(' %s' % tokens[ind])
  print()

Cluster 0:
 role in
 see the
 in this
 to see
 his shop
 his shoulders
 his shoulder
 his shot
 his short
 his shoes

Cluster 1:
 opponents standing
 hurt your
 your opponents
 suggest that
 play with
 prestige ranking
 start competing
 well suggest
 regularly so
 with regularly

Cluster 2:
 an aperture
 the lanterns
 the iso
 while keeping
 lets in
 more noise
 less light
 shutter speed
 then try
 but again

Cluster 3:
 arcanist but
 the smn
 interesting ones
 the xpac
 smn questline
 because started
 flesh out
 be biased
 biased because
 more from

Cluster 4:
 upvoted this
 because would
 advice but
 see some
 this because
 any advice
 would like
 have any
 don have
 like to

Cluster 5:
 map stuff
 programmer but
 lead programmer
 enabled it
 to map
 here like
 stuff out
 think lot
 people around
 the lead

Cluster 6:
 the intimidator
 say skip
 first defiant
 defiant predates
 love bob
 in 05
 05 and
 bob and
 isn anywhere
 honesty the

Cluster 7:
 valid vote
 howard much
 down juli

Could you put a label on the clusters? Do the cluster terms look meaningful?


Let's explore the clustering in more detail by looking at the assignments of posts to the clusters. We'll print out the contents by looking a sample of the top 10 posts per cluster.

In [65]:
# Group the posts by their cluster labels.
clustering = collections.defaultdict(list)
for idx, label in enumerate(kmeans.labels_):
  clustering[label].append(idx)


In [66]:
for cluster, indices in clustering.items():
  print("\nCluster:", cluster, " Num posts: ", len(indices))
  cur_docs = 0
  for index in indices:
    if (cur_docs > 10):
      break
    post_contents = post_vals[index].replace('\n', '')
    print(index, post_keys[index], (post_contents[:75] + '..') if len(post_contents) > 75 else post_contents)
    cur_docs+=1


Cluster: 12  Num posts:  107887
0 https://www.reddit.com/r/100movies365days/comments/1bx6qw/dtx120_87_nashville/ 4/7/13  7/27/12  http://www.imdb.com/title/tt0073440/referenceIt was only a..
1 https://www.reddit.com/r/100movies365days/comments/1bx6qw/dtx120_87_nashville/ I've wanted to watch this for a long time. I was also turned off by the cou..
2 https://www.reddit.com/r/100movies365days/comments/1bx6qw/dtx120_87_nashville/ You strike me as the type who would appreciate it. I would give it a go. Th..
3 https://www.reddit.com/r/100movies365days/comments/1bx6qw/dtx120_87_nashville/ Yeah, I've always heard that Altman was famous for his ensemble casts. But ..
4 https://www.reddit.com/r/100sets/comments/omv7p/male_23_years_old_going_for_100_sets/ **January 16th 3 Sets:** Went out shopping with my grandma as I visited her..
5 https://www.reddit.com/r/100sets/comments/omv7p/male_23_years_old_going_for_100_sets/ grandmas are the best wingmen.
6 https://www.reddit.com/r/100sets/comments/om



*   Is the clustering useful to explore the data?
*   Can you label the clusters now that you've seen some examples?
*   Are these 'good' clusters?

If you have time, try changing the number of clusters - e.g 50 or 100. What do you observe about differences?

Note: KMeans can be somewhat slow to execute exactly. See the MiniBatchKmeans variable above.  If you have time, try clustering using this clustering algorithm.  Do you see any significant differences?



## Summary 


*   We've explored one-hot vector representations of text using several libraries.
*   We've created count, tf-idf, and n-gram representations of text vectors.
*   We've used the vectors to perform unsupervised clustering using scikit learn.
*   Clustered the posts using KMeans


When and why is clustering useful?
*   Clusters can be used as features in a task (text classification and others)
*   Used to perform exploratory data analysis (What's in there?)
*   A fast, easy baseline that scales well and doesn't required labeled data

Next time we'll experiment with additional language prediction tasks using the Vocabulary data structure.


## Please complete the Moodle feedback quiz for this lab

[Quiz for Lab 2](http://moodle2.gla.ac.uk/mod/feedback/view.php?id=827639)